# Add categorical variables

In [ ]:
#| hide 
import sys

In [ ]:
#| hide
IN_COLAB = 'google.colab' in sys.modules

In [ ]:
#| hide
if IN_COLAB:
    %pip install nixtla

In [ ]:
#| hide
if not IN_COLAB:
    from nixtla.utils import colab_badge
    from dotenv import load_dotenv    

In [ ]:
#| echo: false
if not IN_COLAB:
    colab_badge('docs/capabilities/forecast/04_categorical_variables')

[![](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Nixtla/nixtla/blob/main/nbs/docs/capabilities/forecast/04_categorical_variables.ipynb)

## Step 1: Create an instance of `NixtlaClient`

In [ ]:
from nixtla import NixtlaClient

In [ ]:
nixtla_client = NixtlaClient(
    # defaults to os.environ.get("NIXTLA_API_KEY")
    api_key = 'my_api_key_provided_by_nixtla'
)

In [ ]:
#| hide
if not IN_COLAB:
    load_dotenv()
    nixtla_client = NixtlaClient()

## Step 2: Create categorical variables

In [ ]:
import pandas as pd
import datetime
from nixtla.date_features import SpecialDates

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/air_passengers.csv")

In [ ]:
categories_dates = SpecialDates(
    special_dates={
        'christmas_vacations': [datetime.date(year, 12, 1) for year in range(1949, 1960 + 1)],
        'summer_vacations': [datetime.date(year, month, 1) for year in range(1949, 1960 + 1) for month in (6, 7)]
    }
)

dates = pd.date_range('1949-01-01', '1960-12-01', freq='MS')

categories_df = categories_dates(dates).reset_index(drop=True)

## Step 3: Merge categorical features with the dataset

In [ ]:
cat_df = pd.concat([df, categories_df], axis=1)

## Step 4: Forecast

In [ ]:
forecast_df = nixtla_client.forecast(
    df=cat_df, 
    h=24,
    target_col='value',
    time_col='timestamp'
)

INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Inferred freq: MS
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...
